# PEC Monte Carlo simulation

Start by importing MagmaPEC and MagmaPandas and any other packages you want to use. Here we also import Pandas for importing pressure data. For details on the use of MagmaPandas, please see it's [documentation](https://magmapandas.readthedocs.io/en/latest/).

In [1]:
import MagmaPEC as mpc
import MagmaPandas as mp

import pandas as pd

In the next few steps we import all relevant data and set up the melt initial FeO prediction model. These steps are identical to the [PEC model example](https://magmapec.readthedocs.io/en/latest/notebooks/pec_corr.html#)

Import melt inclusion and olivine data:

In [2]:
melt_file = "./data/melt.csv"
olivine_file = "./data/olivine.csv"

melt = mp.read_melt(melt_file, index_col=["name"])
olivine = mp.read_olivine(olivine_file, index_col=["name"])

Import inclusion internal pressures:

In [3]:
pressure_file ="./data/pressure.csv"
pressure = pd.read_csv(pressure_file, index_col = ["name"]).squeeze()

Set up the melt initial FeO prediction model:

In [4]:
wholerock_file = "./data/wholerock.csv"
wholerock = mp.read_melt(wholerock_file, index_col=["name"])

x = wholerock.drop(columns=["FeO"])
FeOi_predict = mpc.FeOi_prediction(x=x, FeO=wholerock["FeO"])

do_not_use = ["MnO", "P2O5", "Cr2O3", "total"]

model_fits = FeOi_predict.calculate_model_fits(exclude=do_not_use)
FeOi_predict.select_predictors(idx=3)

Next, we need to set up the object that handles the random sampling of errors in the Monte Carlo simulation. This is done with the *PEC_MC_parameters* class and it includes the following parameters for error propagation:

- **melt_errors**
        
    propagate errors on melt composition by providing one standard deviation errors per element as a pandas Series (fixed errors for all inclusions) or DataFrame (errors per inclusion).

- **olivine_errors**

    propagate errors on olivine composition by providing one standard deviation errors per element as a pandas Series (fixed errors for all inclusions) or DataFrame (errors per inclusion).

- **FeOi_errors**

    propagate errors on estimate melt initial FeO contents. Fixed errors can be provided either for the whole dataset, or per inclusion. Alternatively, an [FeOi_prediction object](https://magmapec.readthedocs.io/en/latest/notebooks/FeOi.html) can be provided to propagate errors on predictions models.

- **Fe3Fe2**

    Propagate errors on modelled melt Fe<sup>2+</sup>/Fe<sup>3+</sup> ratios. Errors are automatically calculated by MagmaPandas based on the selected model. Pass *True* to this parameter to activate it.

- **Kd**

    Propagate errors on modelled olivine-melt Fe-Mg partition coefficients. Errors are automatically calculated by MagmaPandas based on the selected model. Pass *True* to this parameter to activate it.

By default errors are not propagated - you explicitely need to tell MagmaPEC to do so when initialising the *PEC_MC_parameters* object

In this example we will use all error propagation options, which means we need to provide melt and olivine composition errors. We import these from .csv files containing error data for individual inclusions and olivines. This is just an example with made up errors, normally you would for example use errors from counting statistics.

In [5]:
melt_errors_file = "./data/melt_errors.csv"
olivine_errors_file = "./data/olivine_errors.csv"

melt_errors = pd.read_csv(melt_errors_file, index_col=[0])
olivine_errors = pd.read_csv(olivine_errors_file, index_col=[0])

Make very sure that the elements in the error data have identical sorting to the melt and olivine dataframes, otherwise errors will be applied to the wrong elements. 
We can force this by sorting the columns of the error dataframes (or series) with the *elements* attributes of the melt and olivine MagmaFrames:

In [18]:
melt_errors = melt_errors[melt.elements]
olivine_errors = olivine_errors[olivine.elements]

Here's what they look like:

In [19]:
melt_errors.head()

,SiO2,Al2O3,MgO,CaO,FeO,Na2O,K2O,MnO,TiO2,P2O5,Cr2O3,CO2,H2O,F,S,Cl
PI032-04-01,1.02,0.64,0.14,0.47,0.69,0.10,0.08,0.05,0.18,0.05,0.05,0.17,0.21,0.22,0.10,0.12
PI032-04-02,1.06,0.84,0.29,0.46,0.54,0.05,0.03,0.01,0.06,0.18,0.04,0.24,0.12,0.13,0.12,0.04
PI041-02-02,1.04,0.90,0.09,0.55,0.49,0.22,0.21,0.01,0.16,0.04,0.00,0.15,0.23,0.04,0.17,0.06
PI041-03-01,0.98,0.64,0.10,0.40,0.53,0.12,0.12,0.08,0.01,0.03,0.00,0.05,0.36,0.06,0.14,0.04
PI041-03-03,1.02,0.54,0.31,0.42,0.68,0.18,0.03,0.02,0.11,0.06,0.10,0.18,0.08,0.13,0.06,0.15


In [21]:
olivine_errors.head()

,SiO2,FeO,MgO,NiO,MnO,Al2O3,CaO
PI032-04-01,1.06,0.60,1.22,0.01,0.09,0.08,0.02
PI032-04-02,0.85,0.57,1.28,0.14,0.07,0.24,0.06
PI041-02-02,1.11,0.44,1.30,0.08,0.10,0.01,0.19
PI041-03-01,0.99,0.50,1.31,0.14,0.16,0.15,0.01
PI041-03-03,0.95,0.64,1.21,0.13,0.11,0.10,0.15


Together with the *FeOi_prediction* object, we pass these as arguments to the *PEC_MC_parameters* object. We also set *Fe3Fe2* and *Kd* to *True* in order to propagate their model errors.

In [9]:
mc_parameters = mpc.PEC_MC_parameters(melt_errors=melt_errors, olivine_errors=olivine_errors, FeOi_errors=FeOi_predict, Fe3Fe2=True, Kd=True)

Now we can create the Monte Carlo model with the *PEC_MC* object

In [10]:
pec_mc_model = mpc.PEC_MC(inclusions=melt, olivines=olivine, P_bar=pressure, FeO_target=FeOi_predict, MC_parameters=mc_parameters)

And finally run the Monte Carlo simulation with *n* iterations

In [11]:
pec_mc_model.run(n=30)

Monte Carlo loop
030/030
Equilibrating ... |██████████████████████████████| 100% [10/10] in 2.4s 
Correcting    ... |██████████████████████████████| 100% [10/10] in 7.5s 


Results are stored internally in the following attributes:

- **pec**: pandas DataFrame

    Average PEC extents (mol fraction) of the MC model and their one standard deviation errors.

- **inclusions_corr**: MagmaPandas Melt frame

    Averages of corrected melt inclusion compositions (wt. %)

- **inclusions_stddev**: pandas DataFrame

    One standard deviation errors on inclusions_corr (wt. %)

- **pec_MC**: pandas DataFrame

    PEC extents for individual iterations

- **inclusions_MC**: dictionary of MagmaPandas Melt frames

    corrected melt inclusion compositions for individual iterations.

In [12]:
pec = pec_mc_model.pec
inclusions_corrected = pec_mc_model.inclusions_corr
inclusions_errors = pec_mc_model.inclusions_stddev

pec_mc = pec_mc_model.pec_MC
inclusions_MC = pec_mc_model.inclusions_MC

In [13]:
pec

,pec,stddev
name,,
PI032-04-01,0.074112,0.026094
PI032-04-02,0.094785,0.022353
PI041-02-02,0.005395,0.017765
PI041-03-01,0.123537,0.025827
PI041-03-03,0.116764,0.026163
PI041-05-04,-0.039176,0.012116
PI041-05-06,0.021204,0.017089
PI041-07-01,0.114758,0.024559
PI041-07-02,0.112018,0.025242


In [14]:
inclusions_corrected

,SiO2,Al2O3,MgO,CaO,FeO,Na2O,K2O,MnO,TiO2,P2O5,Cr2O3,CO2,H2O,F,S,Cl
name,,,,,,,,,,,,,,,,
PI032-04-01,49.041533,14.331354,6.954353,9.899754,10.206564,3.687545,0.67605,0.131221,2.459955,0.274938,0.0,0.63133,1.357815,0.147722,0.129243,0.070625
PI032-04-02,48.772518,14.811049,6.937617,9.389899,10.366099,3.495058,0.923374,0.141184,2.606966,0.358199,0.0,0.602788,1.289593,0.105231,0.147095,0.053331
PI041-02-02,48.949259,16.959428,4.894906,9.176832,10.221018,3.814823,1.066538,0.152367,2.803193,0.54367,0.0,0.490849,0.687804,0.065379,0.118129,0.055805
PI041-03-01,45.836723,15.861522,6.946722,11.003291,10.651847,3.364255,1.176459,0.126656,3.144765,0.547736,0.0,0.787663,0.29048,0.090267,0.108384,0.06323
PI041-03-03,45.408553,16.023721,6.777248,11.20739,10.756652,3.432412,1.179158,0.083843,3.27919,0.507582,0.0,0.800742,0.299395,0.066794,0.054486,0.122834
PI041-05-04,47.812576,18.547045,3.850332,9.417566,9.391551,4.610828,1.612232,0.137779,2.498816,0.82507,0.0,0.523988,0.464443,0.124143,0.120603,0.063027
PI041-05-06,46.318225,17.279804,4.730591,8.845196,11.380624,3.991134,1.387067,0.216773,3.6224,0.63211,0.0,0.657887,0.65354,0.098434,0.113315,0.072898
PI041-07-01,45.949339,15.099987,6.904653,9.802128,11.6381,3.170496,1.283141,0.152569,3.574479,0.559676,0.0,0.467367,1.020328,0.097162,0.159399,0.121176
PI041-07-02,45.786739,15.752461,6.76872,10.086839,11.421291,3.200828,1.342617,0.129369,3.555042,0.614985,0.0,0.451238,0.632415,0.073721,0.130749,0.052987


In [15]:
inclusions_errors

,SiO2_stddev,Al2O3_stddev,MgO_stddev,CaO_stddev,FeO_stddev,Na2O_stddev,K2O_stddev,MnO_stddev,TiO2_stddev,P2O5_stddev,Cr2O3_stddev,CO2_stddev,H2O_stddev,F_stddev,S_stddev,Cl_stddev
name,,,,,,,,,,,,,,,,
PI032-04-01,0.806164,0.769978,0.584163,0.383153,0.30792,0.145521,0.085234,0.037743,0.175153,0.051522,0.0,0.166505,0.182574,0.139341,0.084226,0.073432
PI032-04-02,0.573317,0.870612,0.639528,0.3816,0.254705,0.068003,0.035824,0.014295,0.069525,0.186439,0.0,0.199984,0.107172,0.100607,0.102839,0.029055
PI041-02-02,0.793337,0.915537,0.438089,0.54116,0.30809,0.263083,0.216091,0.013652,0.137382,0.048924,0.0,0.154887,0.223755,0.047575,0.121092,0.042945
PI041-03-01,0.585812,0.719726,0.636034,0.367116,0.192405,0.163696,0.09215,0.074867,0.072594,0.026539,0.0,0.04305,0.246391,0.057333,0.105903,0.034658
PI041-03-03,0.438178,0.557913,0.635138,0.39173,0.227489,0.153899,0.036729,0.015564,0.08393,0.051398,0.0,0.17882,0.067996,0.06097,0.047056,0.106356
PI041-05-04,0.765392,0.649262,0.377467,0.338078,0.258012,0.201461,0.120357,0.105528,0.062027,0.03245,0.0,0.208537,0.177754,0.11602,0.010538,0.01045
PI041-05-06,0.873655,0.658949,0.41475,0.249895,0.215053,0.421655,0.192612,0.185138,0.126908,0.049709,0.0,0.185071,0.309085,0.084425,0.062239,0.033213
PI041-07-01,0.607074,0.663775,0.531488,0.389357,0.221299,0.079946,0.037678,0.066026,0.082319,0.019257,0.0,0.058599,0.190424,0.094494,0.024139,0.10463
PI041-07-02,0.559986,0.666933,0.594348,0.355555,0.189418,0.125171,0.152091,0.058101,0.078159,0.013603,0.0,0.168547,0.194762,0.001551,0.041155,0.051149


In [16]:
pec_mc.head()

name,PI032-04-01,PI032-04-02,PI041-02-02,PI041-03-01,PI041-03-03,PI041-05-04,PI041-05-06,PI041-07-01,PI041-07-02,PI052-01-02
iteration,,,,,,,,,,
0,0.07591,0.111282,0.025056,0.10429,0.110593,-0.038781,0.021204,0.102467,0.11107,-0.066554
1,0.025178,0.095149,-0.0223,0.092481,0.095444,-0.036109,0.028844,0.132672,0.098646,-0.082901
2,0.095737,0.121565,0.019937,0.153699,0.153982,-0.03662,0.034276,0.136691,0.128006,-0.071872
3,0.056639,0.07025,0.005277,0.110733,0.142713,-0.053185,0.043803,0.142287,0.107413,-0.081
4,0.086087,0.084303,0.024249,0.120903,0.090856,-0.037054,0.029521,0.127946,0.133596,-0.069655


The dataframes in *inclusions_MC* also have *isothermal_equilibration* and *Kd_equilibration* columns. These columns show if equilibration during stage [1] and [2] respectively was successful. In extreme cases, error propagation of melt and olivine compositions by random sampling can result in melt-olivine pairs that cannot be equilibrated without crystallising more than the entire inclusion or exchanging more Mg or Fe than is present in the inclusion. If that is the case, no corrected compositions are calculated and eiher the isothermal- of Kd-equilibration column is set to *False*.

In [17]:
inclusions_MC["PI032-04-01"].head()

,SiO2,Al2O3,MgO,CaO,FeO,Na2O,K2O,MnO,TiO2,P2O5,Cr2O3,CO2,H2O,F,S,Cl,isothermal_equilibration,Kd_equilibration
iteration,,,,,,,,,,,,,,,,,,
0,49.661598,13.806311,6.824978,10.345639,10.233447,3.989133,0.725367,0.120917,2.43257,0.232457,0.0,0.342075,1.030644,0.054837,0.046573,0.153454,True,True
1,49.445814,13.750238,5.959594,10.575849,10.428247,3.95845,0.649209,0.11835,2.637012,0.27678,0.0,0.75066,1.187604,0.0,0.262194,0.0,True,True
2,49.90605,13.916407,7.209874,9.408774,10.052584,3.736056,0.651988,0.133939,2.261876,0.303991,0.0,0.781169,1.254157,0.038918,0.194462,0.149755,True,True
3,50.291037,14.258685,6.376797,10.075805,9.897942,3.540486,0.769268,0.09858,2.45648,0.319815,0.0,0.644631,1.270473,0.0,0.0,0.0,True,True
4,47.993891,14.26806,7.349929,9.924735,10.330296,3.647482,0.770314,0.227694,2.548839,0.296251,0.0,0.641148,1.687225,0.146429,0.145182,0.022526,True,True
